In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import random
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image

In [2]:
dataDirectory = 'Downloads/DataSet_Split/'
categories = ['cardboard', 'plastic', 'glass', 'metal', 'paper', 'trash']

In [ ]:
for category in categories:
    path = os.path.join(dataDirectory, category) #Path for each folder
    for image in os.listdir(path): #Path for each image of the particular folder
        image_array = cv2.imread(os.path.join(path, image), cv2.IMREAD_UNCHANGED) #Image is unchanged (meaning - RGB image)
        plt.imshow(image_array)
        plt.show()
        break
    break

In [ ]:
#Since, it is a colored image, the array is 2D and the values ranges from 0 to 255 (RGB)

print (image_array)

In [ ]:
for category in categories:
    path = os.path.join(dataDirectory, category) #Path for each folder
    for image in os.listdir(path): #Path for each image of the particular folder
        image_array = cv2.imread(os.path.join(path, image), cv2.IMREAD_GRAYSCALE) #Image is converted into a Gray scale image
        plt.imshow(image_array, cmap = 'gray')
        plt.show()
        break
    break

In [ ]:
#It is a 1D array, as the image is converted into a gray scale image

print (image_array)

In [ ]:
#The data set has images in a highly irregular shape. Normalization is required

image_size = 50

resized_array = cv2.resize (image_array, (image_size, image_size))
plt.imshow(resized_array)
plt.show()

In [3]:
training_data = []
image_size = 50

def create_training_data():
    for category in categories:
        path = os.path.join(dataDirectory, category) #Path for each folder
        class_number = categories.index(category) #assign a number for each class. 
        for image in os.listdir(path): #Path for each image of the particular folder
            try:
                image_array = cv2.imread(os.path.join(path, image), cv2.IMREAD_UNCHANGED) #Image is unchanged (meaning - RGB image)
                resized_array = cv2.resize (image_array, (image_size, image_size)) #Resize the images
                training_data.append([resized_array, class_number]) 
            except Exception as e:
                pass
        

In [4]:
create_training_data()

print(len(training_data))

1977


In [5]:
#Shuffling the dataset, so that the model would learn better

random.shuffle(training_data)

In [ ]:
for samples in training_data[:20]:
    print (samples[1])

In [6]:
features = []
labels = []

for feat, label in training_data:
    features.append(feat)
    labels.append(label)
    
features = np.array(features).reshape(-1, image_size, image_size, 3)

In [ ]:
plt.imshow(features[0])
plt.show()

print (labels[0])


In [7]:
pickle_out = open('features.pickle', 'wb') #w-write, b-binary
pickle.dump(features, pickle_out)
pickle_out.close()

In [8]:
pickle_out = open('labels.pickle', 'wb') #w-write, b-binary
pickle.dump(labels, pickle_out)
pickle_out.close()

In [9]:
pickle_in = open ('features.pickle', 'rb')
features = pickle.load(pickle_in)

In [10]:
pickle_in = open ('labels.pickle', 'rb')
labels = pickle.load(pickle_in)

In [11]:
name = "Plastic_2x64-{}".format (int (time.time()))

tf_dir = os.path.join("logs", name)
tensorBoard = TensorBoard (log_dir =  tf_dir)

model = Sequential()

model.add(Conv2D(64, (3,3), input_shape = features.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation ('relu'))

model.add(Dense(6))
model.add(Activation ('softmax'))

sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile (loss = 'sparse_categorical_crossentropy', 
              optimizer = 'adam',
              metrics = ['accuracy'])

features = np.asarray(features)
labels = np.asarray(labels)

model.fit (features, labels, validation_split = 0.1, epochs = 20, callbacks = [tensorBoard])

Train on 1779 samples, validate on 198 samples
Epoch 1/20
1779/1779 [==============================] - 10s 6ms/sample - loss: 13.9114 - accuracy: 0.2485 - val_loss: 1.7885 - val_accuracy: 0.2323
Epoch 2/20
1779/1779 [==============================] - 8s 4ms/sample - loss: 1.5810 - accuracy: 0.3440 - val_loss: 1.7230 - val_accuracy: 0.2626
Epoch 3/20
1779/1779 [==============================] - 8s 4ms/sample - loss: 1.4316 - accuracy: 0.4087 - val_loss: 1.9275 - val_accuracy: 0.2576
Epoch 4/20
1779/1779 [==============================] - 8s 4ms/sample - loss: 1.2740 - accuracy: 0.5003 - val_loss: 1.7839 - val_accuracy: 0.3535
Epoch 5/20
1779/1779 [==============================] - 8s 5ms/sample - loss: 1.1042 - accuracy: 0.5649 - val_loss: 1.7950 - val_accuracy: 0.3333
Epoch 6/20
1779/1779 [==============================] - 9s 5ms/sample - loss: 0.9755 - accuracy: 0.6290 - val_loss: 2.0462 - val_accuracy: 0.3838
Epoch 7/20
1779/1779 [==============================] - 8s 4ms/sample - los

In [22]:
image_path="Downloads/DataSet_Split/Testing/glass/glass10.jpg"
img = image.load_img(image_path, target_size=(50, 50))
# plt.imshow(img)

img = np.expand_dims(img, axis=0)
img = tf.cast(img, tf.float32)
result=model.predict_classes((img))
print (result)



[3]
